# Creating sources

This notebook will illustrate how to create different kinds of neutron sources for the beamline.

In [ ]:
import numpy as np
import scipp as sc
import tof

Hz = sc.Unit('Hz')

## The ESS source

Sources are characterized by five aspects:

- the number of neutrons in a pulse
- the time at which each neutron is born inside the pulse
- the wavelength of each neutron inside the pulse
- the pulse frequency (or repetition rate)
- the number of pulses

The default way of creating a source is to choose a facility name and a number of neutrons.
Each facility defines time and wavelength probability distributions for the neutrons, as well as a pulse frequency.
By default, a single pulse is generated.

In [ ]:
source = tof.Source(facility='ess', neutrons=1_000_000)
source

To inspect the data in the pulse, we can either look at the source's `data` property

In [ ]:
source.data

or we can plot it with

In [ ]:
source.plot()

## Specifying time and wavelength distributions

It is also possible to create sources with custom time and wavelength distributions.
For this, we need to use the `Source.from_distribution()` method.

### Flat distributions

The `from_distribution` method require two arrays that define the time (`p_time`) and wavelength (`p_wav`) distributions for the neutrons in the pulse.
The array values represent the probabilities, while the associated coordinates represent the values to be sampled from.

We first show how to create flat time and wavelength distributions.


To create a pulse with 1 million neutrons, uniformly distributed in the ranges of 1-3 ms for birth times,
and 1-10 Å for wavelengths, we write:

In [ ]:
time = sc.array(dims=['time'], values=[1.0, 3.0], unit='ms')
p_time = sc.DataArray(
    data=sc.ones(sizes=time.sizes),
    coords={'time': time},
)

wavelength = sc.array(dims=['wavelength'], values=[1.0, 10.0], unit='angstrom')
p_wav = sc.DataArray(
    data=sc.ones(sizes=wavelength.sizes),
    coords={'wavelength': wavelength},
)

source = tof.Source.from_distribution(
    neutrons=1_000_000,
    p_time=p_time,
    p_wav=p_wav,
)
source.plot()

### Custom distributions

Pulses at neutron facilities are rarely flat,
and it is thus useful to be able to supply custom distributions as arrays of probabilities.
The array values represent the probabilities, while the associated coordinates represent the values to be sampled from.

As an example, we create a triangular distribution for the neutron birth times,
and a linearly increasing distribution for the neutron wavelengths
(note that internally a linear interpolation is performed on the original data).

In [ ]:
v = np.arange(30.0)
p_time = sc.DataArray(
    data=sc.array(dims=['time'], values=np.concatenate([v, v[::-1]])),
    coords={'time': sc.linspace('time', 0.1, 6.0, len(v) * 2, unit='ms')},
)
p_wav = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[1.0, 4.0]),
    coords={
        'wavelength': sc.array(dims=['wavelength'], values=[1.0, 4.0], unit='angstrom')
    },
)

source = tof.Source.from_distribution(neutrons=200_000, p_time=p_time, p_wav=p_wav)
source.plot()

Note that the time and wavelength distributions are independent;
a neutron with a randomly selected birth time from `p_time` can adopt any wavelength in `p_wav`
(in other words, the two distributions are simply broadcast into a square 2D parameter space).

## Specifying neutrons manually

Finally, it is possible to simply specify a list of birth times and wavelengths manually to create a pulse via the `from_neutrons` method.

In [ ]:
birth_times = sc.array(
    dims=['event'],
    values=[0.0, 0.1, 0.2, 0.56],
    unit='ms',
)
wavelengths = sc.array(dims=['event'], values=[5.0, 8.0, 11.0, 7.1], unit='angstrom')

source = tof.Source.from_neutrons(birth_times=birth_times, wavelengths=wavelengths)
source.plot()

## Multiple pulses

To make more than one pulse, use the `pulses` parameter:

In [ ]:
tof.Source(facility='ess', neutrons=100_000, pulses=3).plot()

If a custom distribution is supplied, a frequency for the pulse repetition rate must be supplied:

In [ ]:
tof.Source.from_distribution(
    neutrons=200_000, p_time=p_time, p_wav=p_wav, pulses=2, frequency=100.0 * Hz
).plot()

If a source was created from individual neutrons, the same neutrons will be repeated in all the pulses:

In [ ]:
tof.Source.from_neutrons(
    birth_times=birth_times, wavelengths=wavelengths, pulses=3, frequency=500.0 * Hz
).plot()